Part 2 (5 points): Fine-tuning with PEFT and LoRA

In [ ]:
%pip install --quiet transformers==4.37.2 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2 datasets==2.14.7

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.nn.functional as F
import peft
import transformers
from datasets import load_dataset
import random
const_seed = 100

In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 41.2 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.4-cp310-cp310-linux_x86_64.whl size=120038621 sha256=c4ff6919917d4ace36ed5adb65bf056b2572bb395d4a6d7c3c81fdeb6fb2fb87
  Stored in directory: /root/.cache/pip/wheels/18/5b/b5/91951fa1ae1a208051cc3f1bcc55e941d2dfc81509ac80b591
Successfully built flash-attn


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# load llama tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id

# Note: to speed up inference you can use flash attention 2 (https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2)
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2", device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float16,
    attn_implementation="flash_attention_2",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [ ]:
df = load_dataset("google/boolq")

In [ ]:
peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM,
                                      num_virtual_tokens=16) #
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-plac)))

In [ ]:
model.print_trainable_parameters() # Wow so small amount of trainable params

trainable params: 65,536 || all params: 7,241,797,632 || trainable%: 0.000904968673943746


In [ ]:
# creating simple prompt formating
def format_prompt(sample):
    return f'''
    text: {sample['passage']}
    question: {sample['question']}
    answer: {sample['answer']}
    '''

In [ ]:
train = [format_prompt(df["train"][k]) for k in range(0, len(df["train"]))]
valid= [format_prompt(df["validation"][k]) for k in range(0,len(df["validation"]))]

In [ ]:
len(train)

9427

In [ ]:
train_without_true_false = [train[i].replace("True", "").replace("False", "").replace("TRUE", "").replace("FALSE", "").replace("false", "").replace("true", "") for i in range(len(train))]
valid_without_true_false = [valid[i].replace("True", "").replace("False", "").replace("TRUE", "").replace("FALSE", "").replace("false", "").replace("true", "") for i in range(len(valid))]

In [ ]:
tlabel_dataset = Dataset.from_dict({"prompt": train})
vlabel_dataset = Dataset.from_dict({"prompt": valid})
train_dataset = Dataset.from_dict({"prompt": train_without_true_false})
valid_dataset = Dataset.from_dict({"prompt": valid_without_true_false})

In [ ]:
train_labels = [label for label in tlabel_dataset['prompt']]  # 从tlabel_dataset提取标签
valid_labels = [label for label in vlabel_dataset['prompt']]  # 从vlabel_dataset提取标签

# 将标签信息添加到train_dataset和valid_dataset中
train_dataset = Dataset.from_dict({"prompt": train_without_true_false, "completion": train_labels})
valid_dataset = Dataset.from_dict({"prompt": valid_without_true_false, "completion": valid_labels})


In [ ]:
print(train_dataset[0])

{'prompt': '\n    text: Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.\n    question: do iran and afghanistan speak the same language\n    answer: \n    ', 'completion': '\n    text: Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari si

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset


def preprocess_function(examples):
    prompt_text = examples["prompt"]
    completion_text = examples["completion"]

    tokenized_prompt = tokenizer(prompt_text, padding="max_length", truncation=True, max_length=128)

    tokenized_examples = {
        "completion": completion_text  # 将completion文本放入tokenized_examples
    }

    return tokenized_examples



tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_train_dataset[0])

{'prompt': '\n    text: Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.\n    question: do iran and afghanistan speak the same language\n    answer: \n    ', 'completion': '\n    text: Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari si

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/model_output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    logging_dir="/content/drive/My Drive/logs",
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="epoch",
    save_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    gradient_accumulation_steps=2
)

In [ ]:
!pip install trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    packing=True
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Checkpoint destination directory /content/drive/My Drive/model_output/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory /content/drive/My Drive/model_output/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_r

Epoch,Training Loss,Validation Loss
0,2.056900,2.022271
1,2.006100,2.008353


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=784, training_loss=2.0248886079204325, metrics={'train_runtime': 18046.3916, 'train_samples_per_second': 0.348, 'train_steps_per_second': 0.043, 'total_flos': 2.7401048139025613e+17, 'train_loss': 2.0248886079204325, 'epoch': 2.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_path = "/content/drive/My Drive/Mistral_model"
torch.save(model.state_dict(), model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TODO: initialize Trainer and pass train part of our dataset for 2-3 epoches

Note: carefully set max_seq_length and args (that are transformers.TrainingArguments)

TODO: save and check your tuned model. Provide scores on our 20 validation examples and save result to csv file

READ OUTPUT